In [87]:
%run utils.ipynb

In [42]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import dill
%matplotlib inline

In [9]:
import sklearn
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, top_k_accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.feature_extraction import DictVectorizer

In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout, Activation, Input, Embedding, TextVectorization, Reshape, Add, Concatenate, Flatten
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [79]:
def load_test_data(data_path=DATA_PATH):
    _, seg_df = load_user_data(data_path)
    labels = pd.read_csv('challengeToFill.csv').drop(columns=['Unnamed: 0']).T.reset_index(drop=True).T
    y_seg = labels.iloc[:LABELED_USERS, TRAIN_SEG:]
    y_seg = y_seg.to_numpy().flatten()
    y_user = pd.Series([idx//(2*TRAIN_SEG) for idx, _ in enumerate(y_seg)])
    X = seg_df.iloc[:LABELED_USERS, TRAIN_SEG:]
    return X, y_seg, y_user

In [94]:
class Submitter:
    
    def __init__(self, model):
        self.model=model
        
    def evaluate_model(self, X, y_seg, y_user=None, k=3):
        y_user = y_user.to_numpy().flatten() if y_user is not None else None
        try:
            y_pred = self.model.predict_proba(X)
        except:
            y_pred = self.model.predict(X)
        y_pred_seg = np.array([1 if t in p else 0 for t, p in zip(y_user, get_top_k_classes(y_pred, k))]) if y_user is not None else y_pred
        get_metrics(y_seg, y_pred_seg)

In [89]:
X, y_seg, y_user = load_test_data()
X.shape, y_seg.shape,  y_user.shape

((10, 100), (1000,), (1000,))

In [90]:
with open(f'KNeighborsClassifier.dill', 'rb') as f:
    clf = dill.load(f)

In [91]:
clf

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function <lambda> at 0x0000027B9D764670>)),
                ('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.1, min_df=0.01, ngram_range=(1, 3),
                                 sublinear_tf=True)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_neighbors=10, weights='distance'))])

In [98]:
clf = tf.keras.models.load_model('model.h5')

In [99]:
submitter = Submitter(clf)
submitter.evaluate_model(X, y_seg, y_user=None, k=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).